### Este Notebook estara enfocado en el desanidado y normalizacion de los datos de la columna `MISC`. Al terminar continuar con el Notebook 4.

In [1]:
import pandas as pd

In [ ]:
# Leemos la tabla g_sitios que tenemos hasta el momento.
g_sitios = pd.read_parquet(r'C:\Users\mauri\OneDrive\Escritorio\proyectogrupal\data\google_sitios.parquet', engine='pyarrow')

In [ ]:
# Creamos una nueva tabla misc
misc_sitios = g_sitios[['gmap_id', 'MISC']]

# Donde MISC sea NULO no hay Información Se pueden Borrar ya que esta tabla es SOLO MISCELANEOS.
misc_sitios = misc_sitios.dropna(subset=['MISC'])

In [ ]:
# Se Define una Función que convierte los Diccionarios en Listas para poder Aplicar `explode` despues.  Sólo se toman aquellos pares del diccionario donde value is not None
def dict_to_list(diccionario:dict) -> list:
    """ Devuelve el Diccionario de la columna `MISC` y lo retorna
                    como una lista de String "key:Value" Siempre y cuando el 
                    value de la pareja no sea Nulo.

    Args:
        diccionario (dict): diccionario recibido

    Returns:
        list: Lista a devolver
    """
    return [f"{key}: {value}" for key, value in diccionario.items() if value is not None]

In [ ]:
# Aplicamos la funcion definida arriba.
misc_sitios['lista_misc'] = misc_sitios['MISC'].apply(dict_to_list)

# Muestra parcial de la tabla:
misc_sitios.head()

Se borra la columna MISC, se aplica explode al dataframe con respecto a la columna `lista_misc` y luego renombrar la columna para tener lista la tabla

In [ ]:
misc_sitios = misc_sitios.drop(columns=['MISC'])

misc_sitios = misc_sitios.explode('lista_misc')

misc_sitios.rename(columns={'lista_misc': 'misc'}, inplace=True)
misc_sitios.head()

In [ ]:
# Guaradamos la tabla desanidada que se conecta a travez de gmap_id con la tabla principal.
misc_sitios.to_parquet('datasets/google/sitios/google_misc_sitios.parquet')

--- 

### Volver a esta parte del notebook luego de ejecutar el notebook 7.g_alcance donde se quedaron las filas necesarias para proseguir con esta tabla

In [26]:
# Vemos la tabla con las filas actualizadas para actualizar tipos de datos
import pandas as pd
misc = pd.read_parquet(r'C:\Users\mauri\OneDrive\Escritorio\proyectogrupal\data\g_misc_v2.parquet', engine='pyarrow')

In [27]:
pd.set_option('display.max_colwidth', None)
misc

,id_negocio,misc
0,0x89c2f6163ac2864f:0xa4213a2f66c15d8f,Accessibility: ['Wheelchair accessible entrance']
1,0x864c1f10c079a921:0x95710cd26231cc3,Accessibility: ['Wheelchair accessible entrance']
2,0x8824d759c04cb37b:0xef1cb7398b3d692e,Accessibility: ['Wheelchair accessible entrance']
3,0x88d9050582e1e64b:0x33457ba08c64df20,Health & safety: ['Appointment required' 'Mask required' 'Temperature check required'\n 'Staff wear masks' 'Staff get temperature checks'\n 'Staff required to disinfect surfaces between visits']
4,0x88d9050582e1e64b:0x33457ba08c64df20,Service options: ['Online appointments']
...,...,...
82026,0x89b62c2db5856cd3:0x89e0d1a12b77f78a,Accessibility: ['Wheelchair accessible entrance']
82027,0x89b7fdc26ba070ef:0x376e99b8fe4bfc32,Accessibility: ['Wheelchair accessible entrance' 'Wheelchair accessible parking lot']
82028,0x54c0e365e43253f9:0x4366520651be08bb,Accessibility: ['Wheelchair accessible parking lot']
82029,0x54c0e365e43253f9:0x4366520651be08bb,Planning: ['Quick visit']


In [28]:
# Extraer el texto antes del primer ':'
misc['category'] = misc['misc'].str.split(':').str[0]

# Obtener las categorías únicas
unique_categories = misc['category'].unique()
misc

,id_negocio,misc,category
0,0x89c2f6163ac2864f:0xa4213a2f66c15d8f,Accessibility: ['Wheelchair accessible entrance'],Accessibility
1,0x864c1f10c079a921:0x95710cd26231cc3,Accessibility: ['Wheelchair accessible entrance'],Accessibility
2,0x8824d759c04cb37b:0xef1cb7398b3d692e,Accessibility: ['Wheelchair accessible entrance'],Accessibility
3,0x88d9050582e1e64b:0x33457ba08c64df20,Health & safety: ['Appointment required' 'Mask required' 'Temperature check required'\n 'Staff wear masks' 'Staff get temperature checks'\n 'Staff required to disinfect surfaces between visits'],Health & safety
4,0x88d9050582e1e64b:0x33457ba08c64df20,Service options: ['Online appointments'],Service options
...,...,...,...
82026,0x89b62c2db5856cd3:0x89e0d1a12b77f78a,Accessibility: ['Wheelchair accessible entrance'],Accessibility
82027,0x89b7fdc26ba070ef:0x376e99b8fe4bfc32,Accessibility: ['Wheelchair accessible entrance' 'Wheelchair accessible parking lot'],Accessibility
82028,0x54c0e365e43253f9:0x4366520651be08bb,Accessibility: ['Wheelchair accessible parking lot'],Accessibility
82029,0x54c0e365e43253f9:0x4366520651be08bb,Planning: ['Quick visit'],Planning


In [29]:
# Extraer el contenido después de ':' en la columna 'misc'
misc['misc_content'] = misc['misc'].str.split(':').str[1].str.strip()
# Usar una expresión regular para extraer cada texto dentro de comillas simples
misc['misc_content'] = misc['misc'].str.extractall(r"'(.*?)'").groupby(level=0).agg(list)
# Separar la lista en filas
misc = misc.explode('misc_content')
# Muestra de la tabla
misc = misc[['id_negocio', 'category', 'misc_content']]

In [30]:
misc.rename(columns={'category': 'categoria_atributo'}, inplace=True)
misc.rename(columns={'misc_content': 'atributo'}, inplace=True)
misc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 137923 entries, 0 to 82030
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   id_negocio          137923 non-null  object
 1   categoria_atributo  137923 non-null  object
 2   atributo            137881 non-null  object
dtypes: object(3)
memory usage: 4.2+ MB


In [32]:
import os
# Guardamos el nuevo Dataframe g_horarios.
os.makedirs('../../../data', exist_ok=True)
misc.to_parquet('../../../data/g_misc_v3.parquet', index=False)

# Muestra parcial de la tabla y guardado de la informacion.
misc

,id_negocio,categoria_atributo,atributo
0,0x89c2f6163ac2864f:0xa4213a2f66c15d8f,Accessibility,Wheelchair accessible entrance
1,0x864c1f10c079a921:0x95710cd26231cc3,Accessibility,Wheelchair accessible entrance
2,0x8824d759c04cb37b:0xef1cb7398b3d692e,Accessibility,Wheelchair accessible entrance
3,0x88d9050582e1e64b:0x33457ba08c64df20,Health & safety,Appointment required
3,0x88d9050582e1e64b:0x33457ba08c64df20,Health & safety,Mask required
...,...,...,...
82027,0x89b7fdc26ba070ef:0x376e99b8fe4bfc32,Accessibility,Wheelchair accessible parking lot
82028,0x54c0e365e43253f9:0x4366520651be08bb,Accessibility,Wheelchair accessible parking lot
82029,0x54c0e365e43253f9:0x4366520651be08bb,Planning,Quick visit
82030,0x54c0e365e43253f9:0x4366520651be08bb,Service options,In-store shopping
